# K-Means Clustering Tutorial 🎯

This notebook provides a complete implementation and tutorial of the K-Means clustering algorithm from scratch.

## What you'll learn:
- How K-Means algorithm works step by step
- Implementation from scratch in Python
- Visualization of clustering results
- How to use the algorithm for predictions

**Perfect for Google Colab!** Just run each cell sequentially.

## 📦 Import Required Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List

# Set up matplotlib for better plots
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 12

print(" Libraries imported successfully!")

## 🧮 K-Means Algorithm Implementation

Let's implement K-Means from scratch to understand how it works internally.

In [ ]:
class KMeans:
    """K-Means clustering algorithm implementation from scratch."""

    def __init__(self, k: int, max_iters: int = 100, random_state: int = None):
        """
        Initialize K-Means clustering.

        Args:
            k: Number of clusters
            max_iters: Maximum number of iterations
            random_state: Random seed for reproducibility
        """
        self.k = k
        self.max_iters = max_iters
        self.random_state = random_state

    def fit(self, X: np.ndarray) -> 'KMeans':
        """
        Fit K-Means to the data.

        Args:
            X: Data points of shape (n_samples, n_features)

        Returns:
            self: Fitted KMeans object
        """
        if self.random_state:
            np.random.seed(self.random_state)

        n_samples, n_features = X.shape

        # Step 1: Randomly pick initial cluster centers (centroids) from within the range of our data.
        # This makes self.centroids a (k, n_features) array of points between the min and max value of X.
        # To help you understand what this code does, let's use a real example.
        # Suppose X is a set of points like:
        # X = np.array([[1, 2], [2, 3], [3, 2], [8, 7], [7, 8], [8, 8]])
        # X.min() = 1, X.max() = 8, so centroids are initialized as random points in [1, 8] for each feature.
        # For 2 clusters (k=2) in 2D:
        # self.centroids = np.random.uniform(1, 8, (2, 2))
        # Example output for centroids could be:
        # array([[6.25, 3.41],
        #        [2.17, 7.88]])
        #
        # This initialization spreads out k random points across the full data range before fitting.
        self.centroids = np.random.uniform(X.min(), X.max(), (self.k, n_features))

        # Store history for visualization
        self.centroid_history = [self.centroids.copy()]

        for iteration in range(self.max_iters):
            # Step 2: Assign points to closest centroid
            distances = self._calculate_distances(X)
            self.labels = np.argmin(distances, axis=1)

            # Step 3: Update centroids
            new_centroids = np.zeros((self.k, n_features))
            for i in range(self.k):
                if np.sum(self.labels == i) > 0:  # Avoid empty clusters
                    new_centroids[i] = X[self.labels == i].mean(axis=0)
                else:
                    new_centroids[i] = self.centroids[i]  # Keep old centroid

            # Check for convergence
            if np.allclose(self.centroids, new_centroids):
                print(f"✅ Converged after {iteration + 1} iterations")
                break

            self.centroids = new_centroids
            self.centroid_history.append(self.centroids.copy())

        return self

    def _calculate_distances(self, X: np.ndarray) -> np.ndarray:
        """Calculate Euclidean distances from points to centroids."""
        distances = np.zeros((X.shape[0], self.k))
        for i, centroid in enumerate(self.centroids):
            distances[:, i] = np.sqrt(np.sum((X - centroid) ** 2, axis=1))
        return distances

    def predict(self, X: np.ndarray) -> np.ndarray:
        """Predict cluster labels for new data points."""
        distances = self._calculate_distances(X)
        return np.argmin(distances, axis=1)

    def inertia_(self) -> float:
        """Calculate within-cluster sum of squares (WCSS)."""
        wcss = 0
        for i in range(self.k):
            cluster_points = self.X[self.labels == i] if hasattr(self, 'X') else []
            if len(cluster_points) > 0:
                wcss += np.sum((cluster_points - self.centroids[i]) ** 2)
        return wcss

print(" K-Means class implemented successfully!")

## 📊 Generate Sample Data

Let's create some sample data with clear clusters to test our algorithm.

In [ ]:
def generate_sample_data() -> np.ndarray:
    """Generate sample 2D data with clear clusters."""
    np.random.seed(42)

    # Create 3 clusters
    cluster1 = np.random.normal([2, 2], [0.5, 0.5], (30, 2))
    cluster2 = np.random.normal([6, 6], [0.5, 0.5], (30, 2))
    cluster3 = np.random.normal([2, 6], [0.5, 0.5], (30, 2))

    return np.vstack([cluster1, cluster2, cluster3])

# Generate the data
X = generate_sample_data()
print(f" Generated {len(X)} data points with 2 features")
print(f"Data shape: {X.shape}")
print(f"Data range: X-axis [{X[:, 0].min():.2f}, {X[:, 0].max():.2f}], Y-axis [{X[:, 1].min():.2f}, {X[:, 1].max():.2f}]")

## 📊 Visualization Functions

In [ ]:
def visualize_kmeans(X: np.ndarray, kmeans: KMeans, title: str = "K-Means Clustering"):
    """Visualize K-Means clustering results."""
    plt.figure(figsize=(10, 8))

    colors = ['red', 'blue', 'green', 'purple', 'orange']

    # Plot data points
    for i in range(kmeans.k):
        cluster_points = X[kmeans.labels == i]
        plt.scatter(cluster_points[:, 0], cluster_points[:, 1],
                   c=colors[i], alpha=0.6, label=f'Cluster {i+1}', s=50)

    # Plot centroids
    plt.scatter(kmeans.centroids[:, 0], kmeans.centroids[:, 1],
               c='black', marker='x', s=200, linewidths=3, label='Centroids')

    plt.title(title, fontsize=16, fontweight='bold')
    plt.xlabel('Feature 1', fontsize=14)
    plt.ylabel('Feature 2', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

def plot_original_data(X: np.ndarray):
    """Plot the original data without clustering."""
    plt.figure(figsize=(10, 8))
    plt.scatter(X[:, 0], X[:, 1], c='gray', alpha=0.6, s=50)
    plt.title('Original Data (Before Clustering)', fontsize=16, fontweight='bold')
    plt.xlabel('Feature 1', fontsize=14)
    plt.ylabel('Feature 2', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.show()

print(" Visualization functions ready!")

## 👀 Let's Look at Our Data First

In [ ]:
# Plot the original data
plot_original_data(X)
print("Can you see natural clusters forming? ")

## 🚀 Run K-Means Clustering

Now let's apply our K-Means algorithm to find the clusters!

In [ ]:
# Apply K-Means
print("🎯 Applying K-Means with k=3...")
kmeans = KMeans(k=3, random_state=42)
kmeans.fit(X)

print(f"\n Final centroids:")
for i, centroid in enumerate(kmeans.centroids):
    print(f"  Cluster {i+1}: ({centroid[0]:.2f}, {centroid[1]:.2f})")

print(f"\n Cluster assignments:")
unique, counts = np.unique(kmeans.labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"  Cluster {cluster+1}: {count} points")

## 🎨 Visualize the Results

In [ ]:
# Visualize results
visualize_kmeans(X, kmeans, "K-Means Clustering Results (k=3)")
print(" Beautiful clustering! Notice how the algorithm found the natural groups.")

## 🔮 Making Predictions on New Data

Now let's use our trained model to predict which cluster new points belong to.

In [ ]:
# Example of predicting new points
print(" Predicting clusters for new points...")
new_points = np.array([[1, 1], [7, 7], [3, 5], [2.5, 2.5]])
predictions = kmeans.predict(new_points)

print("\n Predictions:")
for i, (point, pred) in enumerate(zip(new_points, predictions)):
    print(f"  Point {point} → Cluster {pred + 1}")

# Visualize predictions
plt.figure(figsize=(12, 8))

colors = ['red', 'blue', 'green', 'purple', 'orange']

# Plot original clustered data
for i in range(kmeans.k):
    cluster_points = X[kmeans.labels == i]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1],
               c=colors[i], alpha=0.6, label=f'Cluster {i+1}', s=50)

# Plot centroids
plt.scatter(kmeans.centroids[:, 0], kmeans.centroids[:, 1],
           c='black', marker='x', s=200, linewidths=3, label='Centroids')

# Plot new predictions
for i, (point, pred) in enumerate(zip(new_points, predictions)):
    plt.scatter(point[0], point[1], c=colors[pred], marker='*', 
               s=300, edgecolors='black', linewidths=2, 
               label=f'New Point {i+1}' if i == 0 else "")
    
    # Add text annotation
    plt.annotate(f'New {i+1}', (point[0], point[1]), 
                xytext=(5, 5), textcoords='offset points', fontweight='bold')

plt.title('K-Means: Original Clusters + New Predictions', fontsize=16, fontweight='bold')
plt.xlabel('Feature 1', fontsize=14)
plt.ylabel('Feature 2', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(" New points are marked with stars and predicted cluster colors!")

## 🧪 Experiment: Different Values of K

Let's see what happens when we use different numbers of clusters.

In [ ]:
# Test different K values
k_values = [2, 3, 4, 5]
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for idx, k in enumerate(k_values):
    # Apply K-Means with different k
    kmeans_k = KMeans(k=k, random_state=42)
    kmeans_k.fit(X)
    
    colors = ['red', 'blue', 'green', 'purple', 'orange']
    
    # Plot data points
    for i in range(k):
        cluster_points = X[kmeans_k.labels == i]
        axes[idx].scatter(cluster_points[:, 0], cluster_points[:, 1],
                         c=colors[i], alpha=0.6, label=f'Cluster {i+1}', s=50)
    
    # Plot centroids
    axes[idx].scatter(kmeans_k.centroids[:, 0], kmeans_k.centroids[:, 1],
                     c='black', marker='x', s=150, linewidths=3)
    
    axes[idx].set_title(f'K-Means with K={k}', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Feature 1')
    axes[idx].set_ylabel('Feature 2')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(" Which K value looks best to you? K=3 seems to match the natural clusters!")

## 📚 Summary & Key Takeaways

### What we learned:

1. **K-Means Algorithm Steps:**
   - Initialize K random centroids
   - Assign points to closest centroid
   - Update centroids to cluster centers
   - Repeat until convergence

2. **Key Properties:**
   - Always converges to a solution
   - Minimizes Within-Cluster Sum of Squares (WCSS)
   - Creates Voronoi partitions

3. **Important Considerations:**
   - Need to choose K beforehand
   - Sensitive to initialization
   - Assumes spherical clusters
   - Works best with similar-sized clusters

### Next Steps:
- Try the "Find Optimal K" methods (Elbow, Silhouette Analysis)
- Experiment with real datasets
- Compare with scikit-learn's implementation
- Explore other clustering algorithms (DBSCAN, Hierarchical)


## 🔍 Bonus: Compare with Scikit-learn

Let's compare our implementation with the professional one!

In [ ]:
# Install scikit-learn if not available (uncomment next line if needed)
# !pip install scikit-learn

from sklearn.cluster import KMeans as SklearnKMeans

# Apply sklearn K-Means
sklearn_kmeans = SklearnKMeans(n_clusters=3, random_state=42, n_init=10)
sklearn_labels = sklearn_kmeans.fit_predict(X)
sklearn_centroids = sklearn_kmeans.cluster_centers_

# Compare results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

colors = ['red', 'blue', 'green']

# Our implementation
for i in range(3):
    cluster_points = X[kmeans.labels == i]
    ax1.scatter(cluster_points[:, 0], cluster_points[:, 1],
               c=colors[i], alpha=0.6, label=f'Cluster {i+1}', s=50)
ax1.scatter(kmeans.centroids[:, 0], kmeans.centroids[:, 1],
           c='black', marker='x', s=200, linewidths=3, label='Centroids')
ax1.set_title('Our K-Means Implementation', fontsize=16, fontweight='bold')
ax1.set_xlabel('Feature 1', fontsize=14)
ax1.set_ylabel('Feature 2', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Sklearn implementation
for i in range(3):
    cluster_points = X[sklearn_labels == i]
    ax2.scatter(cluster_points[:, 0], cluster_points[:, 1],
               c=colors[i], alpha=0.6, label=f'Cluster {i+1}', s=50)
ax2.scatter(sklearn_centroids[:, 0], sklearn_centroids[:, 1],
           c='black', marker='x', s=200, linewidths=3, label='Centroids')
ax2.set_title('Scikit-learn K-Means', fontsize=16, fontweight='bold')
ax2.set_xlabel('Feature 1', fontsize=14)
ax2.set_ylabel('Feature 2', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🎯 Both implementations should give very similar results!")
print(f"Our centroids:\n{kmeans.centroids}")
print(f"\nSklearn centroids:\n{sklearn_centroids}")